In [169]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [170]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append')
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [171]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [172]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST


In [173]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·‘’'']'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@class="article_view"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)

    return full_body

In [174]:
page = 0
max_page = 0
REG_DATE = '20230819'

In [175]:
while(True):
    df_list = []
    response = requests.get(f'http://news.daum.net/breakingnews/digital?page={page}&regDate={REG_DATE}')
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        print(url)
        print(article)
        print('-'*100)
        df_list.append([url, a.text, article])

    if df_list:
        df_10 = pd.DataFrame(df_list)
        df_10.columns = ['URL', 'TITLE', 'ARTICLE']
        db_save(df_10)

    # 페이지 번호 중에서 max 페이지 가져오기
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num
        except:
            pass

    # 마지막 페이지 여부 확인
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page >= max_page):
        break
    else:
        page = page + 1

    time.sleep(1)

https://v.daum.net/v/20230819234647018
 이데일리 강민구 기자 올여름 폭염 특보가 내려지는 등 유독 더운 가운데 전 세계적으로 실제 올 여름이 기후 관측 기록상 가장 더웠던 달이라는 연구 결과가 나왔습니다 미국항공우주국NASA 고다드우주연구소에 따르면 올 7월은 1880년 이후 가장 더웠던 달로 분석됐습니다 앞서 NASA 등이 지난 6월이 관측기록상 가장 더웠던 달이라고 발표한 데 이어 7월도 새로운 기록을 갈아치웠습니다 표면적으로는 엘니뇨동태평양의 해수면 온도가 평년보다 높게 나타나는 현상로 때문일 수 있습니다 엘니뇨나 라니냐의 교체 주기에 따라 지구 기온이 달라질 수 있기 때문입니다 실제 고다드우주연구소는 높은 해수면 온도로 기온도 상승했고 그 원인은 지난 5월부터 시작된 동태평양의 해수면 온도 상승에서 찾았습니다 여기에 온실가스 배출 등이 함께 작용하며 온난화가 심각해지는 것으로 보입니다 기후학자 개빈 슈미트는 지난 달은 우리 기록에서 가장 따뜻한 달로 나타났다며 전 세계의 놀라운 온난화는 주로 인간이 유발한 온실가스 배출에 의해 발생하며 평균 기온의 상승은 전 세계에서 극심한 더위를 유발했다고 했습니다 문제는 NASA 자료에 따르면 온실가스 배출에 따라 인간 주도 온난화가 장기적으로 심화하고 있다는 것입니다 1880년 이후 가장 더웠던 7월 상위 5개 기록은 모두 지난 5년 동안 일어났습니다 앞으로 더 더워질 수 있고 인류의 행동이 필요하다는 것을 뜻합니다 캐서린 캘빈 NASA 수석 과학자는 기후 변화는 전 세계의 사람과 생태계에 영향을 미치고 있으며 온난화는 심화할 것이라고 밝혔습니다 빌 넬슨 NASA 국장도 NASA 분석자료는 글자 그대로 전 세계 수십억 명이 느낀 것을 알려주며 지난달은 기록상 가장 더운 달이 됐다며 과학적으로 이는 분명하며 인류 공동체와 지구를 보호하기 위해 당장 행동해야 한다고 전했습니다 강민구 science1@edailycokr 
-------------------------------------

In [176]:
# db_delete()

In [177]:
news_df = db_select()
news_df

                                        URL  \
0    https://v.daum.net/v/20230819234647018   
1    https://v.daum.net/v/20230819230004921   
2    https://v.daum.net/v/20230819215210587   
3    https://v.daum.net/v/20230819211227263   
4    https://v.daum.net/v/20230819210404188   
..                                      ...   
123  https://v.daum.net/v/20230819014105475   
124  https://v.daum.net/v/20230819004312187   
125  https://v.daum.net/v/20230819002701070   
126  https://v.daum.net/v/20230819002612069   
127  https://v.daum.net/v/20230819001800035   

                                                 TITLE  \
0                  “올해 7월 150년만에 가장 더웠다”…온난화 심각[우주이야기]   
1            [디지털 전략 점검] '쿠키리스' 진짜 도래… 언론사 광고도 제품다워질 때   
2           “유명 연예인도 아이폰 타령” 설마했는데, 삼성 ‘충격’ 사태 결국 터진다?   
3                         T1 "다시 만난 젠지, 이번엔 자기객관화 끝냈다"   
4                   디캐릭, '에듀플러스위크 미래교육박람회'서 디코딩XR 돋보였네   
..                                                 ...   
123  “만화는 끝까지 읽죠?… 전세사기 예방부터 

In [178]:
excel = pd.ExcelWriter('news.xlsx')
news_df.to_excel(excel, 'Daum뉴스', index=False)
excel.save()

<ipython-input-178-298052c84b40>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel.save()
